<a href="https://colab.research.google.com/github/14-vaibhav/pbl_form_corrector/blob/main/SquatFormCorrector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow tensorflow-hub opencv-python numpy


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2
from collections import Counter
from google.colab import files


In [ ]:
model = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
movenet = model.signatures['serving_default']


In [ ]:
def get_keypoints(frame):
    img = tf.image.resize_with_pad(tf.expand_dims(frame, axis=0), 256, 256)
    img = tf.cast(img, tf.int32)
    outputs = movenet(img)
    return outputs['output_0'][0][0]

def angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine = np.dot(ba, bc) / (np.linalg.norm(ba)*np.linalg.norm(bc))
    return np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))


In [ ]:
def analyze_squat(keypoints):
    LEFT_SHOULDER = 5
    LEFT_HIP = 11
    LEFT_KNEE = 13
    LEFT_ANKLE = 15

    shoulder = keypoints[LEFT_SHOULDER][:2]
    hip = keypoints[LEFT_HIP][:2]
    knee = keypoints[LEFT_KNEE][:2]
    ankle = keypoints[LEFT_ANKLE][:2]

    knee_angle = angle(hip, knee, ankle)
    back_angle = angle(shoulder, hip, knee)

    feedback = {
        "depth": "good",
        "back": "good"
    }

    if knee_angle > 100:
        feedback["depth"] = "shallow"

    if back_angle < 20:
        feedback["back"] = "rounded"

    return feedback


In [ ]:
uploaded = files.upload()
video_path = list(uploaded.keys())[0]


TypeError: 'NoneType' object is not subscriptable

In [ ]:
cap = cv2.VideoCapture(video_path)
all_feedback = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    keypoints = get_keypoints(rgb)
    feedback = analyze_squat(keypoints)
    all_feedback.append(feedback)

cap.release()


In [ ]:
from collections import Counter

# --- Aggregate ---
total_frames = len(all_feedback)

depth = Counter([f["depth"] for f in all_feedback])
back = Counter([f["back"] for f in all_feedback])

shallow_ratio = depth["shallow"] / total_frames if total_frames > 0 else 0
rounded_ratio = back["rounded"] / total_frames if total_frames > 0 else 0

# --- Dynamic Scoring ---
score = 100
improvements = []

# Depth penalty (scaled)
depth_penalty = int(shallow_ratio * 40)
score -= depth_penalty

if shallow_ratio > 0.25:
    improvements.append(
        f"Go deeper: insufficient squat depth in {int(shallow_ratio*100)}% of the movement."
    )

# Back posture penalty (scaled)
back_penalty = int(rounded_ratio * 40)
score -= back_penalty

if rounded_ratio > 0.25:
    improvements.append(
        f"Maintain neutral spine: back rounding detected in {int(rounded_ratio*100)}% of the movement."
    )

# Consistency bonus
if shallow_ratio < 0.15 and rounded_ratio < 0.15:
    score += 10

score = max(0, min(score, 100))

# --- Final Report ---
print("🏋️ CampusFit – Squat Form Review\n")
print(f"📊 Form Score: {score}/100\n")

if score >= 85:
    print("🔥 Excellent form. Very low injury risk.")
elif score >= 65:
    print("👍 Acceptable form, but improvements recommended.")
else:
    print("⚠️ Poor form detected. Correct technique before increasing load.")

print("\n🛠 Improvements:")
if improvements:
    for i, tip in enumerate(improvements, 1):
        print(f"{i}. {tip}")
else:
    print("No major issues detected. Keep training!")


🏋️ CampusFit – Squat Form Review

📊 Form Score: 73/100

👍 Acceptable form, but improvements recommended.

🛠 Improvements:
1. Go deeper: insufficient squat depth in 68% of the movement.


In [ ]:
print("🏋️ CampusFit – Squat Form Review\n")

print(f"📊 Form Score: {score}/100\n")

if score >= 85:
    print("🔥 Excellent squat form. Keep it up!")
elif score >= 65:
    print("👍 Decent form, but improvements needed.")
else:
    print("⚠️ Poor form detected. Risk of injury.")

print("\n🛠 Improvements:")
if improvements:
    for i, tip in enumerate(improvements, 1):
        print(f"{i}. {tip}")
else:
    print("No major issues detected. Great job!")


🏋️ CampusFit – Squat Form Review

📊 Form Score: 73/100

👍 Decent form, but improvements needed.

🛠 Improvements:
1. Go deeper: insufficient squat depth in 68% of the movement.
